In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive'

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
test=pd.read_csv(root_path+'/Sarcasm_Headlines_Dataset2.csv',header = 0)
type(test)
test.head()
test.describe


<bound method NDFrame.describe of                                                 headline  ...  tri-gram
0      former versace store clerk sues over secret 'b...  ...         0
1      the 'roseanne' revival catches up to our thorn...  ...         0
2      mom starting to fear son's web series closest ...  ...         0
3      boehner just wants wife to listen, not come up...  ...         0
4      j.k. rowling wishes snape happy birthday in th...  ...         0
...                                                  ...  ...       ...
26704               american politics in moral free-fall  ...         0
26705                            america's best 20 hikes  ...         0
26706                              reparations and obama  ...         0
26707  israeli ban targeting boycott supporters raise...  ...         0
26708                  gourmet gifts for the foodie 2014  ...         0

[26709 rows x 19 columns]>

In [ ]:
pip install senticnet

     |████████████████████████████████| 51.9MB 57kB/s 


In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
# Contradiction in the sentiment score and positive neagtive scores of headlines using senticnet
from senticnet.senticnet import SenticNet
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sn = SenticNet()
lemmatizer = WordNetLemmatizer()
contra=[]
neg=[]
pos=[]
low_pos=[]
med_pos=[]
high_pos=[]
low_neg=[]
med_neg=[]
high_neg=[]
headlines=test['headline'].values


for i in headlines:
  temp_neg=0
  temp_pos=0
  a=i.split()
  for j in range(len(a)):
    a[j]=lemmatizer.lemmatize(a[j])
  
  for i in range(len(a)):
    try:
      concept_info = sn.concept(a[i])
      if (concept_info['polarity_label']=='positive'):
        temp_pos=temp_pos + float (concept_info['polarity_value'])
      if (concept_info['polarity_label']=='negative'):
        temp_neg=temp_neg + float (concept_info['polarity_value'])
    except KeyError:
      temp_neg=temp_neg+0
      temp_pos=temp_pos+0

  temp_neg=temp_neg*-1
  if (temp_pos < 1 and temp_pos >=0):
    low_pos.append(1)
    med_pos.append(0)
    high_pos.append(0)
  if (temp_pos >= 1 and temp_pos < 2):
    low_pos.append(0)
    med_pos.append(1)
    high_pos.append(0)
  if (temp_pos >=2):
    low_pos.append(0)
    med_pos.append(0)
    high_pos.append(1) 
  if (temp_neg < 1 and temp_neg >=0):
    low_neg.append(1)
    med_neg.append(0)
    high_neg.append(0)
  if (temp_neg >= 1 and temp_neg < 2):
    low_neg.append(0)
    med_neg.append(1)
    high_neg.append(0)
  if (temp_neg >= 2):
    low_neg.append(0)
    med_neg.append(0)
    high_neg.append(1) 
  if (temp_pos != 0 and temp_neg !=0):
    contra.append(1)
  else:
    contra.append(0)    

  neg.append(temp_neg)
  pos.append(temp_pos)





/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
test['sum_neg']=neg
test['sum_pos']=pos
test['neg_low']=low_neg
test['neg_med']=med_neg
test['neg_high']=high_neg
test['pos_low']=low_pos
test['pos_medium']=med_pos
test['pos_high']=high_pos
test['contra']=contra
test.to_csv(root_path+'/Sarcasm_Headlines_Dataset2.csv',index=False)

In [ ]:
#punctuations in the headlines


import string

punc_med=[]
punc_low=[]
punc_high=[]
for j in headlines:
  count=0
  Sentence=j
  for i in Sentence:
    if i in string.punctuation:
      count=count+1
  if (count==0):
    punc_low.append(1)
    punc_med.append(0)
    punc_high.append(0)
  if (count >= 1 and count <=3):
    punc_low.append(0)
    punc_med.append(1)
    punc_high.append(0)
  if (count >=4):
    punc_low.append(0)
    punc_med.append(0)
    punc_high.append(1)

        


In [ ]:
test['punct_low']=punc_low
test['punc_med']=punc_med
test['punc_high']=punc_high
test.to_csv(root_path+'/Sarcasm_Headlines_Dataset2.csv',index=False)

In [ ]:
$#capital letters in headlines

import re

cap_med=[]
cap_low=[]
cap_high=[]


for i in headlines:
  count_capital=0
  line = i
  capital = re.findall(r'(\b[A-Z]([a-z])*\b)',line)
  count_capital=len(capital)
  if (count_capital==0):
    cap_low.append(1)
    cap_med.append(0)
    cap_high.append(0)
  if (count_capital >= 1 and count_capital <=3):
    cap_low.append(0)
    cap_med.append(1)
    cap_high.append(0)
  if (count_capital >=4):
    cap_low.append(0)
    cap_med.append(0)
    cap_high.append(1)


In [ ]:
test['capital_low']=cap_low
test['capital_med']=cap_med
test['capital_high']=cap_high
test.to_csv(root_path+'/Sarcasm_Headlines_Dataset2.csv',index=False)

In [ ]:
#bigrams and trigrams in headlines


import nltk
from nltk import bigrams
from nltk import trigrams
import collections

tri=[]
bi=[]
for i in headlines:
  res=0
  res2=0
  line=i
  line=line.split()
  big=bigrams(line)
  big=list (big)
  tig=trigrams(line)
  tig=list (tig)
  res=[item for item, count in collections.Counter(big).items() if count > 1]
  res2=[item for item, count in collections.Counter(tig).items() if count > 1]
  if (len(res) > 0):
    bi.append(1)
  elif (len(res)==0):
    bi.append(0)  
  if (len(res2) > 0):
    tri.append(1)
  elif (len(res2)==0):
    tri.append(0)   


In [ ]:
test['bi-gram']=bi
test['tri-gram']=tri
test.to_csv(root_path+'/Sarcasm_Headlines_Dataset2.csv',index=False)




In [ ]:
import numpy as np
test.columns
features=test[['sum_pos','sum_neg','contra','pos_low','pos_medium','pos_high','neg_low','neg_med','punct_low','punc_med','punc_high']]

X=np.asarray(features)

y=np.asarray(test['is_sarcastic'])

X[0:5]

array([[0.875, 0.64 , 1.   , 1.   , 0.   , 0.   , 1.   , 0.   , 0.   ,
        1.   , 0.   ],
       [1.17 , 2.76 , 1.   , 0.   , 1.   , 0.   , 0.   , 0.   , 0.   ,
        1.   , 0.   ],
       [1.716, 0.66 , 1.   , 0.   , 1.   , 0.   , 1.   , 0.   , 0.   ,
        1.   , 0.   ],
       [0.797, 0.   , 0.   , 1.   , 0.   , 0.   , 1.   , 0.   , 0.   ,
        1.   , 0.   ],
       [1.901, 0.   , 0.   , 0.   , 1.   , 0.   , 1.   , 0.   , 0.   ,
        1.   , 0.   ]])

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=41)

y_train.shape

(18696,)

In [ ]:
from sklearn import svm
classifier=svm.SVC(kernel='linear',gamma='auto',C=2)
classifier.fit(X_train,y_train)
y_predict=classifier.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
from sklearn import metrics
print(classification_report(y_test,y_predict))
print(metrics.confusion_matrix(y_test, y_predict))


              precision    recall  f1-score   support

           0       0.62      0.64      0.63      4488
           1       0.52      0.49      0.51      3525

    accuracy                           0.58      8013
   macro avg       0.57      0.57      0.57      8013
weighted avg       0.57      0.58      0.57      8013

[[2878 1610]
 [1790 1735]]


In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train,y_train)

y_predictNB=clf.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_predictNB))
print(metrics.confusion_matrix(y_test, y_predictNB))


              precision    recall  f1-score   support

           0       0.61      0.78      0.68      4488
           1       0.56      0.37      0.45      3525

    accuracy                           0.60      8013
   macro avg       0.59      0.57      0.56      8013
weighted avg       0.59      0.60      0.58      8013

[[3484 1004]
 [2228 1297]]


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

lsvc = LinearSVC()
lsvc.fit(X_train,y_train)
y_predictLSVC=lsvc.predict(X_test)
print(classification_report(y_test,y_predictLSVC))
print(metrics.confusion_matrix(y_test, y_predictLSVC))


              precision    recall  f1-score   support

           0       0.61      0.79      0.69      4488
           1       0.57      0.36      0.44      3525

    accuracy                           0.60      8013
   macro avg       0.59      0.57      0.57      8013
weighted avg       0.59      0.60      0.58      8013

[[3533  955]
 [2250 1275]]


In [ ]:

# predicting new headline as sarcastic or not

user_sum_pos=0
user_sum_neg=0
user_pos_low=0
user_contra=0
user_pos_medium=0
user_pos_high=0
user_neg_low=0
user_neg_med=0
user_punct_low=0
user_punc_med=0
user_punc_high=0

a=input ()
b=a
a=a.split()

for j in range(len(a)):
  
  a[j]=lemmatizer.lemmatize(a[j])
for i in range(len(a)):
  try:
    concept_info = sn.concept(a[i])
    if (concept_info['polarity_label']=='positive'):
      temp_pos=temp_pos + float (concept_info['polarity_value'])
    if (concept_info['polarity_label']=='negative'):
      temp_neg=temp_neg + float (concept_info['polarity_value'])
  except KeyError:
    temp_neg=temp_neg+0
    temp_pos=temp_pos+0

temp_neg=temp_neg*-1
if (temp_pos < 1 and temp_pos >=0):
  user_pos_low=1
  user_pos_medium=0
  user_pos_high=0
if (temp_pos >= 1 and temp_pos < 2):
  user_pos_low=0
  user_pos_medium=1
  user_pos_high=0
if (temp_pos >=2):
  user_pos_low=0
  user_pos_medium=0
  user_pos_high=1
if (temp_neg < 1 and temp_neg >=0):
  user_neg_low=1
  user_neg_medium=0
  user_neg_high=0
if (temp_neg >= 1 and temp_neg < 2):
  user_neg_low=0
  user_neg_medium=1
  user_neg_high=0
if (temp_neg >= 2):
  user_neg_low=0
  user_neg_medium=0
  user_neg_high=1
if (temp_pos != 0 and temp_neg !=0):
  user_contra=1
else:
  user_contra=0 
user_sum_pos=temp_pos
user_sum_neg=temp_neg


count=0
Sentence=b
for i in Sentence:
  if i in string.punctuation:
    count=count+1
if (count==0):
  user_punct_low=1
  user_punc_med=0
  user_punc_high=0
if (count >= 1 and count <=3):
  user_punct_low=0
  user_punc_med=1
  user_punc_high=0
if (count >=4):
  user_punct_low=0
  user_punc_med=0
  user_punc_high=1  

user_test=[[user_sum_pos,user_sum_neg,user_contra,user_pos_low,user_pos_medium,user_pos_high,user_neg_low,user_neg_med,user_punct_low,user_punc_med,user_punc_high]]
user_predict=classifier.predict(user_test)
print(user_predict)


he is a killer
[1]
